# 🎨 Conditional Image Generation with cGANs

While standard GANs learn to generate realistic images from a dataset, they offer no control over *what kind* of image is generated. The output is entirely random.

**Conditional GANs (cGANs)** solve this by introducing a **condition** (`y`) to the generation process. This condition, which could be a class label, text, or a set of attributes, is fed into both the generator and the discriminator.

- 👨‍🎨 **The Conditional Generator** learns to create 'fake' images that not only look real but also match the given condition `y`.
- 🕵️‍♂️ **The Conditional Discriminator** learns to determine if an image is real *and* if it matches its corresponding condition `y`.

This allows us to direct the image generation process. For this notebook, we'll use the CelebA dataset and condition the GAN on facial attributes like 'Smiling', 'Male', and 'Blond_Hair' to generate specific types of faces.

![GANs vs CGANs Image](https://i.imgur.com/PNRcz5N.png) 
<p align = "center">
Fig.1 - GAN vs. CGAN 
(<a href="https://itchef.ru/articles/341494/">
source
</a>)
</p>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import pandas as pd

from torchvision.transforms import Compose, ToTensor, Lambda, Resize, Normalize
from torchsummary import summary
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from tqdm import tqdm
import numpy as np
from PIL import Image
import kagglehub
import os

## ⚙️ Hyperparameters and Setup
Here we define the key parameters for our cGAN model and training process.

In [ ]:
EPOCHS = 5 # Training for longer can improve results, I found 150 the best for this task.
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
Z = 100 # Latent dimension
IMAGE_SIZE = 64
NUM_ATTRIBUTES = 5 # Number of conditional attributes

## 🖼️ Loading the CelebA Dataset with Attributes

We'll use the CelebA (CelebFaces Attributes) dataset, which is perfect for this task because it includes a large collection of celebrity faces, each annotated with 40 binary attributes.

### 🔹 **Dataset Details:**
- **Source**: `jessicali9530/celeba-dataset` from Kaggle
- **Content**: Over 200,000 celebrity images with attribute annotations.
- **Conditioning**: We will select 5 key attributes to condition our model on: `Male`, `Smiling`, `Young`, `Eyeglasses`, and `Blond_Hair`.
- **Preprocessing**: Images are resized to `64x64` and pixel values are normalized to `[-1, 1]` to match the generator's `Tanh` output.

In [ ]:
# Define transforms to resize and normalize the images
transform_steps = Compose([
    Resize((IMAGE_SIZE, IMAGE_SIZE)), 
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize to [-1, 1]
])

In [ ]:
# Custom dataset class for CelebA faces and attributes
class CelebADataset(Dataset):
    def __init__(self, root_dir, transform=None, max_samples=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Load the main attributes file
        attr_file = os.path.join(root_dir, 'list_attr_celeba.csv')
        self.attr_df = pd.read_csv(attr_file)
        
        # Define the 5 key attributes we'll use for conditioning
        self.selected_attributes = ['Male', 'Smiling', 'Young', 'Eyeglasses', 'Blond_Hair']
        
        # The image directory is nested, so we define the full path
        self.img_dir = os.path.join(root_dir, 'img_align_celeba', 'img_align_celeba')
        
        self.image_files = []
        self.attributes = []
        
        count = 0
        # Iterate through the attribute file to get image names and their labels
        for idx, row in tqdm(self.attr_df.iterrows(), total=len(self.attr_df)):
            if max_samples and count >= max_samples:
                break
                
            img_filename = row['image_id']
            img_path = os.path.join(self.img_dir, img_filename)

            # Only add the file if it actually exists
            if os.path.exists(img_path):
                self.image_files.append(img_path)
                # The attributes are stored as -1 and 1. We convert them to 0 and 1.
                attrs = [(row[attr] + 1) // 2 for attr in self.selected_attributes]
                self.attributes.append(attrs)
                count += 1

    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        # Get the attributes and convert them to a tensor
        attributes = torch.tensor(self.attributes[idx], dtype=torch.long)

        # Open the image and convert to RGB
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        # Return both the image and its corresponding attributes
        return image, attributes

In [ ]:
# Download the CelebA dataset from Kaggle
path = kagglehub.dataset_download("jessicali9530/celeba-dataset")

# Create the dataset object. We use a subset for faster training.
celeba_dataset = CelebADataset(root_dir=path, transform=transform_steps, max_samples=40000)
attribute_names = celeba_dataset.selected_attributes

print(f"\nDataset size: {len(celeba_dataset)}")
print(f"Selected attributes: {attribute_names}")

In [ ]:
# Create the DataLoader
train_dataloader = DataLoader(celeba_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

### 🔹 Visualizing the Data
Let's look at some examples from the dataset to see the images and their associated attributes.

## Building the Conditional GAN (cGAN) Models

Our cGAN consists of two neural networks. The key difference from a standard GAN is that both networks now take the conditional attributes as an additional input. We will follow best practices from the DCGAN paper to stabilize training:

- **Weight Initialization**: Initialize weights from a Normal distribution with `mean=0`, `stdev=0.02`.
- **Activations**: Use `LeakyReLU` to prevent dying gradients.
- **Optimizer**: Use the `Adam` optimizer with `beta1=0.5`.
- **Attribute Handling**: We use `nn.Embedding` layers to convert our binary attributes (0 or 1) into dense vectors, which are easier for the network to process.

--- 

### 👨‍🎨 The Conditional Generator

The Generator's job is to create believable images that match a given set of attributes.

**Architecture:**
1.  **Input**: It takes two inputs: a 100-dimensional noise vector (`Z`) and a 5-element attribute vector.
2.  **Embedding**: Each of the 5 attributes is passed through its own `nn.Embedding` layer. The resulting embedding vectors are concatenated together.
3.  **Concatenation**: The latent noise vector `Z` is concatenated with the final attribute embedding vector.
4.  **Upsampling**: This combined vector is fed through a series of `ConvTranspose2d` layers to upsample it into a `64x64` color image.
5.  **Output**: A `Tanh` activation function ensures the output image pixels are in the range `[-1, 1]`, matching our input data.

--- 

### 🕵️‍♂️ The Conditional Discriminator

The Discriminator's job is to classify an image as real or fake, *given* a set of attributes.

**Architecture:**
1.  **Input**: It takes two inputs: a `64x64x3` image and the 5-element attribute vector.
2.  **Image Path**: The image is passed through a series of `Conv2d` layers to extract features and downsample it.
3.  **Attribute Path**: The attributes are passed through `nn.Embedding` layers, just like in the generator.
4.  **Concatenation**: The flattened feature vector from the image is concatenated with the attribute embedding vector.
5.  **Output**: This combined vector is passed through fully connected layers to produce a single `Sigmoid` output between 0 (fake or mismatched) and 1 (real and matched).

### Model Summaries

### 🎯 Loss Function and Optimizers

- **Loss**: We use **Binary Cross-Entropy Loss** (`BCELoss`), which is ideal for the discriminator's binary classification task (real/matched vs. fake/mismatched).
- **Optimizers**: We use the `Adam` optimizer for both networks with a `beta1` of 0.5, as recommended for stable GAN training.

## 🚀 Training the cGAN

The training process alternates between training the discriminator and the generator. The condition `y` (our attribute vector) is passed to both networks at each step.

### Step 1: Train the Discriminator 🕵️‍♂️
The discriminator's goal is to correctly classify real images that match their conditions, and reject fake images.
$$ \nabla_{\theta_d} [\log D(x|y) + \log(1 - D(G(z|y)))] $$
- We feed it a batch of **real images** with their **real attributes** and train it to classify them as `real` (label=1).
- We then generate a batch of **fake images** using the generator (given the same real attributes) and train the discriminator to classify them as `fake` (label=0).

### Step 2: Train the Generator 👨‍🎨
The generator's goal is to fool the discriminator into thinking its fake images are real and match the given attributes.
$$ \nabla_{\theta_g} \log(D(G(z|y))) $$
- We generate a new batch of **fake images** with their corresponding attributes.
- We pass them through the discriminator, but this time, we use `real` labels (label=1) for the loss calculation. This way, the backpropagated gradients update the **generator's weights** to produce images that the discriminator is more likely to classify as real.

### 📈 Training Loss Visualization

Plotting the losses helps us understand the training dynamics. Ideally, the two losses should reach an equilibrium, indicating that neither model is overpowering the other. 
- **Discriminator Loss (D)**: If this loss drops to zero, the generator is failing to produce convincing fakes, and training has stalled.
- **Generator Loss (G)**: We want to see this loss stay competitive with the discriminator's. If it grows too high, the generator is being beaten too easily.

## ✨ Conditional Generation

Now that training is complete, we can use our generator to create new, unique faces by providing it with random noise and our desired set of attributes.

### 🤔 Discussion

The results show that the cGAN is capable of generating realistic faces that often reflect the requested attributes. For example, images conditioned on 'Blond_Hair' tend to have lighter hair, and those conditioned on 'Smiling' often feature a smile. The model isn't perfect, and sometimes the attributes are subtle or mixed, but the conditioning clearly influences the output.

Challenges with cGANs are similar to standard GANs:
- **Training Instability**: The balance between the generator and discriminator is still delicate.
- **Mode Collapse**: The generator might still produce a limited variety of faces for a given condition.
- **Attribute Entanglement**: The model might learn spurious correlations in the data (e.g., associating lipstick with being 'Female', even if not explicitly told to). The quality of the dataset's labels is crucial.

Despite these challenges, cGANs are a powerful tool for controllable generation and form the basis for many advanced models like StyleGAN and image-to-image translation networks.

## 🧪 What to Try Next

- **Train for Longer**: Increasing the number of epochs can significantly improve the quality and coherence of the generated images.
- **Explore Attribute Combinations**: Generate images with specific, interesting attribute combinations. What does a 'Male, Smiling, Blond_Hair, Eyeglasses' face look like according to the model?
- **Latent Space Interpolation**: Create two noise vectors, `z1` and `z2`, and a fixed attribute vector `y`. Interpolate between `z1` and `z2` and see how the generated face changes while keeping the attributes constant.
- **Attribute Interpolation**: Fix the noise vector `z` but interpolate between two different attribute vectors, `y1` and `y2`. For example, morph a non-smiling face into a smiling one.

### Contributed by: Ali Habibullah.
